### The second team
### DS-SBERKZ Final Project
https://github.com/rinatomarov/Car-price-prediction-Teamwork

In [3]:
import numpy as np 
import pandas as pd 
import sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string
from nltk.util import ngrams
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import catboost
import xgboost as xgb
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [5]:
DIR = 'for_training/'
RANDOM_SEED = 42
VAL_SIZE   = 0.2
sample_submission = pd.read_csv(DIR+'sample_submission.csv')
data = pd.read_csv(DIR+'data2201.csv') 
target = pd.read_csv(DIR+'yy2201.csv')

In [10]:
data

,Unnamed: 0,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,leather,carter,ABS,airbags,immob,central_locking,on_board_computer,cruise_control,climat_control,multi_rudder,power_steering,light_and_rain_sensors,сarbon_body_kits,rear_diffuser_rkp,door_closers,rear_view_camera,amg,bi_xenon_headlights,from_salon,alloy_wheels,parking_sensors,dents,roof_with_panoramic_view,bT_length,Nalog,years_old,age_intervals,sample
0,0,9,8,14,1.2,105.0,0,74000,2013,713,5,2014,3,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2625.0,7,1,0
1,1,9,8,16,1.6,110.0,0,60563,2017,713,5,2017,2,2,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,7,2750.0,4,1,0
2,2,9,8,13,1.8,152.0,0,88000,2013,957,5,2014,3,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,6840.0,7,1,0
3,3,9,8,7,1.6,110.0,0,95000,2013,713,5,2014,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2750.0,7,1,0
4,4,9,8,1,1.8,152.0,0,58536,2008,713,5,2012,0,2,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,7,6840.0,9,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231346,231346,16,11,14,2.4,170.0,0,120000,2000,859,4,2003,0,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7650.0,18,3,1
231347,231347,20,11,16,2.0,160.0,0,230000,1995,1026,5,1998,2,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,7200.0,23,4,1
231348,231348,16,11,12,2.3,131.0,0,100000,1983,70,4,1985,2,6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,4585.0,36,5,1
231349,231349,16,11,16,1.6,109.0,0,245000,1999,856,4,2001,2,6,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2725.0,20,3,1


In [17]:
data = data.drop('Unnamed: 0', axis=1 ,inplace = True)
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)
y = target['price']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

# **Boosting Algorithms**



 **Extreme Gradient Boosting Algorithm.**

> Result in Kaggle submissions = 8.02996



In [ ]:
import xgboost as xgb

In [ ]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.01 ,max_depth=26, alpha=1,
                          n_estimators=15000,tree_method='gpu_hist')


In [ ]:
xg_reg.fit(X_train, np.log(y_train))

XGBRegressor(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=25, min_child_weight=1, missing=None, n_estimators=15000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, tree_method='gpu_hist',
             verbosity=1)

In [ ]:
predict_test = np.exp(xg_reg.predict(X_test)) 
predict_submission = np.exp(xg_reg.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 10.29%


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_xgboost22fin.csv', index=False)


In [ ]:
import pickle

**GradientBoostingRegressor**

> Result in Kaggle submissions = 10.37510

In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.01, max_depth=24, n_estimators=1000)
gb.fit(X_train, np.log(y_train))

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls',
                          max_depth=24, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_2201gb.csv', index=False)


# StackingRegressor

Result in Kaggle submissions = 10.63027

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor


In [ ]:
import xgboost as xgb

In [ ]:
estimators = [
    ('xg_reg', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5,
                          learning_rate= 0.01 ,max_depth=15, alpha=1,
                          n_estimators=5000)),
    ('gb', GradientBoostingRegressor(min_samples_split=2, learning_rate=0.01, max_depth=15,
                                     n_estimators=5000))
]

In [ ]:
reg5 = StackingRegressor(estimators=estimators)
reg5.fit(X_train, np.log(y_train))


In [ ]:
predict_test = np.exp(reg5.predict(X_test))
predict_submission = np.exp(reg5.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_2316st.csv', index=False)
            

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

# ExtraTreesRegressor 
Kaggle submission result: 8.48


In [ ]:
etr = ExtraTreesRegressor(n_estimators=500, random_state=42)

In [ ]:
etr.fit(X_train, np.log(y_train))

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=500, n_jobs=None, oob_score=False,
                    random_state=42, verbose=0, warm_start=False)

In [ ]:
predict_test = np.exp(etr.predict(X_test))
predict_submission = np.exp(etr.predict(X_sub))


In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.02%


In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submissions_etr2.csv', index=False)

# LGBMRegressor 
Result in Kaggle submissions: 11.91138 

In [22]:
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


In [25]:
base_lgbmr = lgb.LGBMRegressor()


In [26]:
param_grid2 = {
    "learning_rate": [0.15],
    "max_depth": [8],
    "n_estimators": [1500,1800],
    "num_leaves": [25,27],
    'reg_alpha': [0,0.001,0.01],
    'reg_lambda': [0,0.001,0.01]
}
gscv_lgbm2 = GridSearchCV(estimator=base_lgbmr, param_grid=param_grid2, n_jobs=-1, verbose=5, cv=4)

In [28]:
gscv_lgbm2.fit(X_train, np.log(y_train))

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 50.1min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.15], 'max_depth': [8],
                         'n_estimators': [1500, 1800], 'num_leaves': [25, 27],
                         'reg_alpha

In [29]:
predict_test = np.exp(gscv_lgbm2.predict(X_test))
predict_submission = np.exp(gscv_lgbm2.predict(X_sub))


In [30]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.39%


In [31]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submissions_lgbm.csv', index=False)

# RandomForestRegressot
Result in Kaggle submissions: 14.38993


In [32]:
rfr = RandomForestRegressor(n_estimators=1000, max_depth = 15, max_features = 'log2', 
                            random_state = 42, oob_score=True,) 

In [33]:
rfr.fit(X_train, np.log(y_train))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=True,
                      random_state=42, verbose=0, warm_start=False)

In [34]:
predict_test = np.exp(rfr.predict(X_test))
predict_submission = np.exp(rfr.predict(X_sub))


In [35]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 14.13%


In [36]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_rf.csv', index=False)
            

# Catboost
Result in Kaggle submissions: 12.47514

In [40]:
pip install catboost

     |████████████████████████████████| 65.8MB 56kB/s 


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [42]:
cb = CatBoostRegressor(iterations = 5000, random_seed = 42, eval_metric='MAPE', 
                       custom_metric=['R2', 'MAE'],silent=True ) 
cb.fit(X_train, np.log(y_train),eval_set=(X_test, np.log(y_test)),verbose_eval=0,use_best_model=True)

In [43]:
predict_test = np.exp(cb.predict(X_test))
predict_submission = np.exp(cb.predict(X_sub))


In [44]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.90%


In [45]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_catboost.csv', index=False)
            

# TransformedTargetRegressor

Kaggle score = 11.17624

In [ ]:
#Notebook with logs - https://github.com/rinatomarov/Car-price-prediction-Teamwork/blob/master/models/rf%2BTransformedTargetRegressor.ipynb
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_regression
import numpy as np

rf = RandomForestRegressor(bootstrap= True,
  max_depth = 80,
 min_samples_leaf = 3,
 min_samples_split = 8,
 n_estimators = 1000)
log_rf = TransformedTargetRegressor(rf, func = np.log, inverse_func=np.exp)
#params = {'regressor__n_estimators': [10,100,1000]}
#gscv = GridSearchCV(log_rf, param_grid=params,refit = True)
#gscv.best_params_
log_rf.fit(X_train, y_train)

In [ ]:
y_pred = log_rf.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

# Blending

#### Точность модели по метрике MAPE: 27.69%
#### Точность модели по метрике MAPE: 35.92%
#### Точность модели по метрике MAPE: 14.68%

In [ ]:
from numpy import hstack
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


# Get a list of base models
def get_models():
	models = list()
	models.append(('lr', LinearRegression()))
	models.append(('knn', KNeighborsRegressor()))
	models.append(('cart', DecisionTreeRegressor()))
	models.append(('svm', SVR()))
	return models

print('Train: %s, Test: %s' % (X_train.shape, X_test.shape))

# create the base models
models = get_models()

for name, model in models:
  model.fit(X_train, np.log(y_train))
  predict_test = np.exp(model.predict(X_test))
  predict_submission = np.exp(model.predict(X_sub))
  print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
  sample_submission['price'] = predict_submission
  sample_submission.to_csv('submission_blending.csv', index=False

# Stacking

**stacking.ipynb**
Модель с логами: https://github.com/rinatomarov/Car-price-prediction-Teamwork/tree/master/models 
### *MAPE: 12.47%*

In [ ]:
X_pred = X_sub.values
X = X.values
y = y.values

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def compute_metric(regr, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    return np.round(mape(y_test, y_test_pred)*100, 3)

def compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred):    
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
    
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)
    
    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(X_pred)
    
    return X_meta_train, X_meta_test, X_meta_pred

In [ ]:
def generate_meta_features2(regr_s, X_train, X_test, y_train, cv, X_pred):
   
    features = [compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred) for regr in tqdm(regr_s)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from tqdm.notebook import tqdm
from sklearn.base import clone

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features2([
    RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                          max_features='sqrt',max_depth=77, bootstrap=True, random_state=42),
    AdaBoostRegressor(random_state=42),
    ExtraTreesRegressor(random_state=42),
    RandomForestRegressor(random_state=42)], X_train, X_test, y_train, cv, X_pred)

regr = RandomForestRegressor(
 n_estimators=300,
 min_samples_split=2,
 min_samples_leaf=1,
 max_features='sqrt',
 max_depth=77,
 bootstrap=True, 
 random_state=42)

print(f'Stacking MAPE = {compute_metric(regr, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test, y_test=y_test)}%')